## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [3]:
import glob
import numpy as np
import scipy as sp
import scipy.io
import scipy.signal as sig
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold,train_test_split 
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from tqdm import tqdm 
import warnings
warnings.filterwarnings("ignore")

In [49]:


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]



def LoadTroikaRefFile(ref_file):
    data = sp.io.loadmat(ref_file)['BPM0']
    return data

def BandpassFilter(signal, pass_band=(40/60,240/60), fs=125):
      
    b, a = sp.signal.butter(2, pass_band, btype='bandpass', fs = fs)
    return sp.signal.filtfilt(b, a, signal)


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)




def extract_features(ppg, accx, accy, accz, fs = 125):   
    """
    Create features from the data
    
    Returns:
        PPG and ACC features
    """
    
    mn_x = np.mean(accx)
    std_x = np.std(accx)
    energy_x = np.sum(np.square(accx - np.mean(accx)))
    
    mn_y = np.mean(accy)
    std_y = np.std(accy)
    energy_y = np.sum(np.square(accy - np.mean(accy)))
    
    mn_z = np.mean(accz)
    std_z = np.std(accz)
    energy_z = np.sum(np.square(accz - np.mean(accz)))
                   
    corr_xy = sp.stats.pearsonr(accx, accy)[0]
    
    mn_ppg = np.mean(ppg)
    std_ppg = np.std(ppg)
    
                      
                      
                      
    # magnitude of accelometer
    acc=np.sqrt(accx**2 + accy**2 + accz**2)
    # frequencies and coefficients for both ppg and acceleormeter magnitude 
    fft_acc=np.fft.rfft(acc,6*len(acc))
    freqs_acc=np.fft.rfftfreq(6*len(acc),1/fs)
    fft_ppg=np.fft.rfft(ppg,6*len(ppg))
    freqs_ppg=np.fft.rfftfreq(6*len(ppg),1/fs)
    #bandpass 
    fft_ppg[freqs_ppg <= 40/60] = 0.0
    fft_ppg[freqs_ppg >= 240/60] = 0.0
        
    # getting peaks from the largest frequencies 
    #1) If the ppg peaks does not match with the acc peaks, have that in your result.
    #2) If the ppg peaks match with the acc peaks, 
    #look for the second max peak of ppg.
    #3) Sometimes, the second peak of ppg also match with the acc peak, 
    #in such case have that in your result.
    max_acc_freq=np.argmax(np.abs(fft_acc))
    max_ppg_freq=np.argmax(np.abs(fft_ppg))
    acc_peak=freqs_acc[max_acc_freq]
    ppg_peak=freqs_ppg[max_ppg_freq]
    if acc_peak==ppg_peak:
        fft_ppg=np.delete(fft_ppg,max_ppg_freq)
        freqs_ppg=np.delete(freqs_ppg,max_ppg_freq)
        max_acc_freq=np.argmax(np.abs(fft_acc))
        max_ppg_freq=np.argmax(np.abs(fft_ppg))
        acc_peak=freqs_acc[max_acc_freq]
        ppg_peak=freqs_ppg[max_ppg_freq]
    return np.array([ppg_peak, acc_peak,mn_x,std_x,energy_x,
                    mn_y,std_y,energy_y,mn_z,std_z,energy_z,corr_xy,mn_ppg,std_ppg])




def ensemble_model():       
    """
    Train Regressor model
    
    Returns:
        Trained model
    """ 
    fs = 125
    window_length = 8
    window_shift = 2
    
    # Load filenames through LoadTroikaDataset
    data_fls, ref_fls = LoadTroikaDataset()
    features, labels, signals = [], [], []
    
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
        ref=LoadTroikaRefFile(ref_fl)
        features, labels, signals = [], [], []
        left_s = (np.cumsum(np.ones(min(len(ppg), len(ref))) * fs * window_shift) - fs * window_shift).astype(int)
        right_s = left_s + fs * window_length
        for i in range(len(left_s)):
            left, right = left_s[i], right_s[i]
            ppg_w=BandpassFilter(ppg[left:right])
            accx_w=BandpassFilter(accx[left:right])
            accy_w=BandpassFilter(accy[left:right])
            accz_w=BandpassFilter(accz[left:right])
            features.append(extract_features(ppg_w, accx_w, accy_w, accz_w))
            labels.append(ref[i])
            signals.append([ppg_w, accx_w, accy_w, accz_w])
    
    features, labels = np.array(features), np.array(labels)
    model = AdaBoostRegressor(n_estimators=400)
    model.fit(features,labels)
    
    return model

def RunPulseRateAlgorithm(data_fl, ref_fl):
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    ref=LoadTroikaRefFile(ref_fl)
    print(f" len of ref ={len(ref)}")
    features, labels, signals = [], [], []
    # Load ground truth heart rate
    fs=125
    window_length=8
    window_shift=2 
    left_s = (np.cumsum(np.ones(min(len(ppg), len(ref))) * fs * window_shift) - fs * window_shift).astype(int)
    right_s = left_s + fs * window_length
    for i in range(len(left_s)):
        left, right = left_s[i], right_s[i]
        ppg_w=BandpassFilter(ppg[left:right])
        accx_w=BandpassFilter(accx[left:right])
        accy_w=BandpassFilter(accy[left:right])
        accz_w=BandpassFilter(accz[left:right])
        features.append(extract_features(ppg_w, accx_w, accy_w, accz_w))
        
        labels.append(ref[i])
        signals.append([ppg_w, accx_w, accy_w, accz_w])
    
    features, labels = np.array(features), np.array(labels)
    print(f" len of labels {len(labels)}")
    model= ensemble_model()
    
    # Compute pulse rate estimates and estimation confidence.
    errors, confidence = [], []
    
    for i in range(len(signals)):
        feature, label = features[i], labels[i]
        ppg, accx, accy, accz = signals[i]
        est = model.predict(np.reshape(feature, (1, -1)))[0]
        
        n = len(ppg) * 3
        freqs = np.fft.rfftfreq(n, 1/fs)
        fft = np.abs(np.fft.rfft(ppg,n))
        fft[freqs <= 40/60.0] = 0.0
        fft[freqs >= 240/60.0] = 0.0
          # Max magnitude frequency
        est_fs = est / 55.0
        fs_win = 30  / 60.0
        fs_win_e = (freqs >= est_fs - fs_win) & (freqs <= est_fs +fs_win)
        conf = np.sum(fft[fs_win_e])/np.sum(fft)
        
        errors.append(np.abs((est-labels[i])))
        confidence.append(conf)
    print(f" len of errors ={len(errors),len(confidence)} \n ")

    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    return np.mean(errors), np.mean(confidence)

In [50]:
Evaluate()

 len of ref =148
 len of labels 148
 len of errors =(148, 148) 
 
 len of ref =148
 len of labels 148
 len of errors =(148, 148) 
 
 len of ref =140
 len of labels 140
 len of errors =(140, 140) 
 
 len of ref =107
 len of labels 107
 len of errors =(107, 107) 
 
 len of ref =146
 len of labels 146
 len of errors =(146, 146) 
 
 len of ref =146
 len of labels 146
 len of errors =(146, 146) 
 
 len of ref =150
 len of labels 150
 len of errors =(150, 150) 
 
 len of ref =143
 len of labels 143
 len of errors =(143, 143) 
 
 len of ref =160
 len of labels 160
 len of errors =(160, 160) 
 
 len of ref =149
 len of labels 149
 len of errors =(149, 149) 
 
 len of ref =143
 len of labels 143
 len of errors =(143, 143) 
 
 len of ref =146
 len of labels 146
 len of errors =(146, 146) 
 


14.585164226544865

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** 
Purpose of code is to build a  machine learning model trained to estmiate pulse from ppg and accelerometer data.
This is a functional code consituting of the following functions:
    * LoadTroikaDataset : loads the files containing our data
    * LoadTroikaDataFile : attains the ppg and accelerometer signal data from the loaded files
    * LoadTroikaRefFile : attains the reference data from loaded files 
    * BandpassFilter : performs bandpass filter to remove  undesirable frequencies outside the defined bandwidth in our function, the allowed bandwidth is from 40 to 240 bpm  ( notice it is divided by 60 to convert unit) 
    * extract_features: extract features from ppg and accleremoter data to be used in model , including peaks of fourier transform ( highest frequencies), mean of signal , and standard deviations of signals. Note the acceleromter magnitude is calculated before performing fourier transform also note the peaks are extracted using a heuritic approach in which the ppg peak should not match the acc peak
    * ensemble_model : trains ensemble model , random forest, and trains it on given features and labels
    * RunPulseRateAlgorithm : given files , loads signals and reference using LoadTroikaDataFile() and LoadTroikaRefFile() respecitively , it then performs windowing technique on all signals in which a window of 10 seconds is taken for every signal , features is extracted and labels is obtained from the reference array, eventually having features and labels arrays in which model is trained using ensemble_model(). Function then calculates errors(MAE) and confidence after predicting pulse using trained model and returns mean error array and mean confidence array 
    * Evaluate : given files,it loads files and uses RunPulseRateAlgorithm() to get errors and confidence arrays for all files and stacks them in one array
    * AggregateErrorMetric : given stacked array of errors and confidence , it computes an aggregate error metric based on confidence estimates to return MAE at 90 % confidence

> - **Data Description** 
 * Data trained and tested on PPG and acceloremter data recorded from wearable device from 12 subjects during fast running at the peak speed of 15 km/h (note there is hidden extra test data that is why all data was used for training), the reference data or labels of our data was obtained from ECG 
 * Data can be improved by having more subjects and by having more features about subjects , like age, gender, or smoking status
> - **Algorithhm Description** will include the following:
>   - Algorithm trains a random forest on  extracted features from ppg and acceleromter signals , features include , ppg maximum frequency that is different than accelormeter , accelerometer maximum frequency and means and std of ppg and acc signals
>   - PPG sensor are light sensors that basically detect light shined from LEDS on the other side of the wrist, if perfusion is low , alot of of light will pass and the photodetector will read a hight signal.During ventricular sytole( which is basically the R wave in the ECG that is ideally used to measure pulse)  , blood is pumped from the heart to the peripheral ciruclation and thus pefusion at wrist increase, blood is a thick liquid with cells and plasma proteins so it blocks much light to photodetector, thus smaller signal. Using this concept we can estimate pulse from PPG , one problem though , perfusion of blood in wrist is also dependent on movement of the arm so data from accelormeter should be used to distinguish chnages of perfusion due to the cardiac cycle and changes due to the arm movement.
>   - algorithim will like fail in extreme exercise or exercises very dependent on hand movement as it will become harder to distguish change in perfusion due to cardiac cycle
> - **Algorithm Performance** - Model was trained without  a test set which might have made it less generalisable and this was due to the fact the data is relatively small, however a mean absolute error of 14.5
Your write-up goes here...

-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 